Note: for using the widgets inside jupyterlab, you need an appropriate environment with nodejs and the widegetextensions install. 
In order to develop the notebook, the following environment was used:

- conda create -n secanalyzing python==3.10
- conda activate secanalyzing
- pip install jupyterlab
- jupyter labextension install @jupyter-widgets/jupyterlab-manager
- pip install secfsdstools
- pip install ipywidgets

In [1]:
# Basic import to support interactive widgets in notebooks
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import pandas as pd
pd.set_option('display.max_rows', 500) # ensure that all rows are shown

In [2]:
# imports from the secfsdstools package
from secfsdstools.e_read.searching import IndexSearch
from secfsdstools.e_read.companyreading import CompanyReader
from secfsdstools.e_read.reportreading import ReportReader
from secfsdstools.e_read.zipreportreading import ZipReportReader
from secfsdstools.d_index.indexing import ReportZipIndexer

In [10]:
# initialize the search class
search = IndexSearch.get_index_search()

# create a list with all known forms
forms_list = sorted(search.dbaccessor.read_all_indexreports_df().form.unique().tolist())
stmt_list = ['BS', 'CF', 'CI', 'CP', 'EQ', 'IS', 'SI', 'UN']

In [4]:
# a simple way to find the cik for a company.
# e.g., start typing apple and watch the list get filtered
@interact(search_string=widgets.Text(value=''))
def search_cik(search_string):
    print(search_string)
    result_df = search.find_company_by_name(search_string)
    display(result_df)
    if len(result_df) > 0:
        print(result_df.cik.tolist()[0])

interactive(children=(Text(value='', description='search_string'), Output()), _dom_classes=('widget-interact',…

In [5]:
# using a cik number, you can filter for all filed reports. Choose the type of report you want to see by selecting the entries in the forms widget.
# e.g., use apples cik '320193' to see an overview of the reports that have were filed by apple
@interact
def reports(cik=widgets.Text(value='0'), forms=widgets.SelectMultiple(options=forms_list, rows=6, value=['10-K', '10-Q']), rows=[10, 25, 50, 100]):
    reader = CompanyReader.get_company_reader(cik=int(cik))
    reports = reader.get_all_company_reports_df(forms=list(forms))
    display(reports.head(rows))

interactive(children=(Text(value='0', description='cik'), SelectMultiple(description='forms', index=(4, 8), op…

In [24]:
@interact
def reports(adsh=widgets.Text(value='0'), stmts=widgets.SelectMultiple(options=stmt_list, rows=6, value=['BS', 'IS', 'CF']), rows=[10, 25, 50, 100, 200], invert_negated=widgets.Checkbox()):
    if adsh=='0':
        display(invert_negated)
        display('Nothing selected - Note: loading may take a few seconds')
        return
    display('loading...')
    reader = ReportReader.get_report_by_adsh(adsh=adsh)

    # get some key infos of the report
    submission_data = {k:v for k,v in reader.submission_data().items() if k in ['cik', 'adsh', 'name', 'cityba', 'form', 'period', 'filed']}

    report_data = reader.merge_pre_and_num()
    
    # we only look at data for the submitted period, meaning we don't show data from the previous period
    report_data = report_data[report_data.ddate==submission_data['period']]
    
    # sort the data in a meaningful way
    report_data = report_data.sort_values(['adsh', 'coreg', 'ddate', 'stmt', 'report', 'line', ])
    
    # filter for the selected statements
    report_data = report_data[report_data.stmt.isin(stmts)]
    
    if invert_negated:
        report_data.loc[report_data.negating==1, 'value'] = -report_data.value
    
    # use a meaningful column order
    report_data = report_data[['adsh', 'coreg', 'ddate', 'stmt', 'report', 'line', 'tag', 'version', 'uom', 'value', 'negating', 'plabel', 'qtrs', 'footnote', 'inpth', 'rfile']]
    
    # create and display the url on which the report is published on sec.gov, so that it can directly be opened    
    url = ReportZipIndexer.URL_PREFIX + str(submission_data['cik']) + '/' + submission_data['adsh'].replace('-','') + '/' + submission_data['adsh'] + '-index.htm'
    display(url)
    
    # display the key submission data of the report
    display(submission_data)
    
    # display the data of the report
    display(report_data.head(rows))

interactive(children=(Text(value='0', description='adsh'), SelectMultiple(description='stmts', index=(0, 5, 1)…